# House Price CV & Baseline Model

In [ ]:
import seaborn as sns

from house_price_utils import *

setup_notebook()

In [ ]:
data, data_test = load_data()
data = data[sorted(data)]
X_raw = data.drop(columns=["SalePrice"])
y = data.SalePrice

In [ ]:
full_data = pd.concat([X_raw, data_test])
features = group_features(full_data)
full_clean_data = clean(full_data, features)
X = full_clean_data[lambda x: x.index.isin(X_raw.index)]
X_test = full_clean_data[lambda x: x.index.isin(data_test.index)]

In [ ]:
X.shape, X_test.shape

In [ ]:
selected_features = make_mi_scores(X, y, "auto").index[-10:].tolist()
selected_features

# Baseline Model
 Given the large number of categorical variables, let's start with random forest so we don't have to do any further encoding.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate

In [ ]:
model = RandomForestRegressor(random_state=0)

In [ ]:
scores = cross_validate(
    model,
    X,
    y,
    cv=10,
    scoring="neg_mean_absolute_error",
    return_train_score=True,
)

In [ ]:
-scores["train_score"].mean(), scores["train_score"].std()

In [ ]:
-scores["test_score"].mean(), scores["test_score"].std()

## Observations
* Model is overfitting.


In [ ]:
try:
    from sklearn.model_selection import ValidationCurveDisplay

    ValidationCurveDisplay.from_estimator(
        model,
        X,
        y,
        param_name="n_estimators",
        param_range=[50, 100, 200, 400, 800],
        scoring="neg_mean_absolute_error",
    )
except ImportError:
    pass

# Submit

In [ ]:
model.fit(X, y)

In [ ]:
create_submission(model, X_test)